# Persona Based SocialBot

In [ ]:
!python -m spacy download en

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 223.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [ ]:
! pip install -r  /content/requirements.txt

In [ ]:
!pip install transformers==2.5.1

In [ ]:
!pip install rouge

In [ ]:
!pip install --upgrade pip
!git clone https://github.com/google-research/bleurt.git
%cd bleurt
!pip install .

fatal: destination path 'bleurt' already exists and is not an empty directory.
/content/bleurt
Processing /content/bleurt
  Preparing metadata (setup.py) ... done
  Created wheel for BLEURT: filename=BLEURT-0.0.2-py3-none-any.whl size=16456761 sha256=5b628fe2457ccbcd5b050a4bb4bf6159b7d8e3ce2014b517996ef2b5944f92e8
  Stored in directory: /tmp/pip-ephem-wheel-cache-6skvv3fw/wheels/15/d1/f6/2ef9b6f78af64ad606636904e6bdc4cfae694df53f1f6f887d
Successfully built BLEURT
  Attempting uninstall: BLEURT
    Found existing installation: BLEURT 0.0.2
    Uninstalling BLEURT-0.0.2:
      Successfully uninstalled BLEURT-0.0.2


In [ ]:
%cd ..

/content


## Libraries


In [ ]:
import json
from datetime import datetime
import socket
from transformers import cached_path
import os
import math
import logging
from pprint import pformat
from collections import defaultdict
from itertools import chain
import nltk
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge
import string
import re
from bleurt import score

import torch

from torch.utils.data import DataLoader, TensorDataset
from ignite.engine import Engine, Events,DeterministicEngine
from ignite.handlers import ModelCheckpoint
from ignite.metrics import Accuracy, Loss, MetricsLambda, RunningAverage
from ignite.contrib.handlers import ProgressBar, PiecewiseLinear
from transformers import (AdamW, OpenAIGPTDoubleHeadsModel, OpenAIGPTTokenizer,OpenAIGPTLMHeadModel,GPT2Tokenizer, GPT2LMHeadModel)
from ignite.contrib.handlers.tensorboard_logger import TensorboardLogger
from transformers import WEIGHTS_NAME,CONFIG_NAME
import os
import logging
from ignite.contrib.handlers.tensorboard_logger import TensorboardLogger
import random
import torch.nn.functional as F

import warnings

In [ ]:
dataset_path = "/content/train_data.json"
val_path = "/content/test_data.json"
max_history = 2
lr = 6.25e-5
n_epochs = 3
device = "cuda"

In [ ]:
SPECIAL_TOKENS = ["<bos>", "<eos>", "<speaker1>", "<speaker2>", "<pad>"]
ATTR_TO_SPECIAL_TOKEN = {'bos_token': '<bos>', 'eos_token': '<eos>', 'pad_token': '<pad>',
                         'additional_special_tokens': ['<speaker1>', '<speaker2>']}
MODEL_INPUTS = ["input_ids", "mc_token_ids", "lm_labels", "mc_labels", "token_type_ids"]
PADDED_INPUTS = ["input_ids", "lm_labels", "token_type_ids"]

## Required functions for  dataset preparation, training and evaluation

In [ ]:
def average_distributed_scalar(scalar, args):
      return scalar

In [ ]:
def pad_dataset(dataset, padding=0):
    max_l = max(len(x) for x in dataset["input_ids"])
    for name in PADDED_INPUTS:
        l = []
        for x in dataset[name]:
          if name!="lm_labels":
            l.append(x + [padding]*(max_l-len(x)))
          else:
            l.append(x + [-100]*(max_l-len(x)))
        dataset[name] = l  
    return dataset

In [ ]:
def add_special_tokens_(model, tokenizer):
    orig_num_tokens = len(tokenizer.encoder)
    num_added_tokens = tokenizer.add_special_tokens(ATTR_TO_SPECIAL_TOKEN) 
    if num_added_tokens > 0:
        model.resize_token_embeddings(new_num_tokens=orig_num_tokens + num_added_tokens)

In [ ]:
def tokenize(dataset):
        for i in dataset:
          conversations = len(dataset[i])
          for l in range(conversations):
              persona = dataset[i][l]["personality"]
              for p in range(len(persona)):
                persona[p] = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(persona[p]))
              dataset[i][l]["personality"] = persona
              utterances = dataset[i][l]["utterances"]
              for u in range(len(utterances)):
                temp = utterances[u]
                candidates = temp["candidates"]
                for c in range(len(candidates)):  
                  candidates[c] = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(candidates[c]))
                
                history = temp["history"]
                for h in range(len(history)):  
                  history[h] = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(history[h]))
                temp["candidates"] = candidates
                temp["history"] = history
                utterances[u] = temp
        return dataset

In [ ]:
def make_input_representation(persona, history, reply, tokenizer, lm_labels=False, with_eos=True):
    bos, eos, speaker1, speaker2 = tokenizer.convert_tokens_to_ids(SPECIAL_TOKENS[:-1])
    sequence = [[bos] + list(chain.from_iterable(persona))] + history + [reply + ([eos] if with_eos else [])]
    s1 = []
    s1 = [sequence[0]]
    for i in range(1,len(sequence)):
      if i%2!=0:
        s1.append([speaker1]+sequence[i])
      else:
        s1.append([speaker2]+sequence[i])
    sequence = s1
   

    instance = {}
    instance["input_ids"] = list(chain.from_iterable(sequence))
    s1 = []
    for i in range(len(sequence)):
      for j in range(len(sequence[i])):
        if i%2==0:
          s1.append(speaker1)
        else:
          s1.append(speaker2)
    instance["token_type_ids"] = s1


    instance["mc_token_ids"] = len(instance["input_ids"]) - 1
    instance["lm_labels"] = [-100] * len(instance["input_ids"])
    if lm_labels:
        instance["lm_labels"] = ([-100] * sum(len(s) for s in sequence[:-1])) + [-100] + sequence[-1][1:]
    return instance

In [ ]:
def get_data_loaders(tokenizer):
    input_file = cached_path(dataset_path)
    with open(input_file, "r", encoding="utf-8") as f:
        dataset = json.loads(f.read())
    data = tokenize(dataset)
    datasets = {"train": defaultdict(list), "valid": defaultdict(list)}
    for dataset_name, dataset in data.items():
        num_candidates = len(dataset[0]["utterances"][0]["candidates"])
        for dialog in dataset:
            persona = dialog["personality"].copy()
            
            for utterance in dialog["utterances"]:
                history = utterance["history"][-(2*max_history+1):]
                for j, candidate in enumerate(utterance["candidates"][-num_candidates:]):
                    lm_labels = bool(j == num_candidates-1)
                    instance = make_input_representation(persona, history, candidate, tokenizer, lm_labels)
                    for input_name, input_array in instance.items():
                        datasets[dataset_name][input_name].append(input_array)
                datasets[dataset_name]["mc_labels"].append(num_candidates - 1)
                datasets[dataset_name]["n_candidates"] = num_candidates
            
    tensor_datasets = {"train": [], "valid": []}
    for dataset_name, dataset in datasets.items():
        dataset = pad_dataset(dataset, padding=tokenizer.convert_tokens_to_ids(SPECIAL_TOKENS[-1]))
        for input_name in MODEL_INPUTS:
            tensor = torch.tensor(dataset[input_name])
            if input_name != "mc_labels":
                tensor = tensor.view((-1, datasets[dataset_name]["n_candidates"]) + tensor.shape[1:])
            tensor_datasets[dataset_name].append(tensor)
    train_dataset, valid_dataset = TensorDataset(*tensor_datasets["train"]), TensorDataset(*tensor_datasets["valid"])
    train_loader = DataLoader(train_dataset, sampler=None, batch_size=1, shuffle=False)
    valid_loader = DataLoader(valid_dataset, sampler=None, batch_size=1, shuffle=False)

    return train_loader, valid_loader,datasets

In [ ]:
def top_filtering(logits, top_k=0.):
    threshold=-float('Inf')
    filter_value=-float('Inf')
    assert logits.dim() == 1
    top_k = min(top_k, logits.size(-1))
    if top_k > 0:
        indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        logits[indices_to_remove] = filter_value

    indices_to_remove = logits < threshold
    logits[indices_to_remove] = filter_value

    return logits

In [ ]:
def sample_sequence(personality, history, tokenizer, model,current_output=None):
    special_tokens_ids = tokenizer.convert_tokens_to_ids(SPECIAL_TOKENS)
    if current_output is None:
        current_output = []
    for i in range(20):
        instance = make_input_representation(personality, history, current_output, tokenizer, with_eos=False)

        input_ids = torch.tensor(instance["input_ids"], device=device).unsqueeze(0)
        token_type_ids = torch.tensor(instance["token_type_ids"], device=device).unsqueeze(0)

        logits = model(input_ids, token_type_ids=token_type_ids)
        if isinstance(logits, tuple):
            logits = logits[0]
        logits = logits[0, -1, :] 
        logits = top_filtering(logits, top_k=0)
        probs = F.softmax(logits, dim=-1)
        prev = torch.multinomial(probs, 1)
        
        if prev.item() in special_tokens_ids:
            break
        current_output.append(prev.item())

    return current_output

In [ ]:
def preprocess_text(out,input):
  out = " ".join(out.split())
  input = " ".join(input.split())
  out = out.translate(str.maketrans("","", string.punctuation))
  input = input.translate(str.maketrans("","", string.punctuation))
  out = out.strip()
  input = input.strip()
  out = re.sub(r'\d+', '', out)
  input = re.sub(r'\d+', '', input)
  print(out)
  return input,out

In [ ]:
tokenizer_class = OpenAIGPTTokenizer 
tokenizer = tokenizer_class.from_pretrained("openai-gpt")

model_class = OpenAIGPTDoubleHeadsModel
model = model_class.from_pretrained("openai-gpt")
model.to(device)

add_special_tokens_(model, tokenizer)
optimizer = AdamW(model.parameters(), lr=lr, correct_bias=True)

train_loader, val_loader,dataset = get_data_loaders(tokenizer)

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


In [ ]:
criterion = torch.nn.CrossEntropyLoss(ignore_index=-100)

scheduler = PiecewiseLinear(optimizer, "lr", [(0, lr), (n_epochs * len(train_loader), 0.0)])

In [ ]:
final_loss = []
count = 0
def fine_tuning():
  temp = []
  temp2 = []
  def update(engine, batch):
    model.train()
    batch = tuple(input_tensor.to(device) for input_tensor in batch)
    input_ids, mc_token_ids, lm_labels, mc_labels, token_type_ids = batch
    (lm_loss), (mc_loss), *_ = model(
        input_ids, token_type_ids=token_type_ids, mc_token_ids=mc_token_ids,
        mc_labels=mc_labels, lm_labels=lm_labels
    )
    loss = (lm_loss  + mc_loss ) / 4
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
    if engine.state.iteration % 4 == 0:
        optimizer.step()
        optimizer.zero_grad()
    return loss.item()

  trainer = Engine(update)

  def inference(engine, batch):
      model.eval()
      with torch.no_grad():
          batch = tuple(input_tensor.to(device) for input_tensor in batch)
          input_ids, mc_token_ids, lm_labels, mc_labels, token_type_ids = batch
          
          lm_logits, mc_logits, *_ = model(
              input_ids, token_type_ids=token_type_ids, mc_token_ids=mc_token_ids,
          )
          lm_logits_flat_shifted = lm_logits[..., :-1, :].contiguous().view(-1, lm_logits.size(-1))
          lm_labels_flat_shifted = lm_labels[..., 1:].contiguous().view(-1)
          loss_t = criterion(lm_logits_flat_shifted,lm_labels_flat_shifted)
          temp.append(math.exp(loss_t))
          temp2.append(loss_t)
          if len(temp) == 5:
            result = sum(temp)/5
            print(result)
            print(sum(temp2)/5)
            temp.clear()
            temp2.clear()
          return (lm_logits_flat_shifted, mc_logits), (lm_labels_flat_shifted, mc_labels)
  evaluator = Engine(inference)


  trainer.add_event_handler(Events.EPOCH_COMPLETED, lambda _: evaluator.run(val_loader))

  scheduler = PiecewiseLinear(optimizer, "lr", [(0, lr), (n_epochs * len(train_loader), 0.0)])
  trainer.add_event_handler(Events.ITERATION_STARTED, scheduler)

  RunningAverage(output_transform=lambda x: x).attach(trainer, "loss")


  pbar = ProgressBar(persist=True)
  pbar.attach(trainer, metric_names=["loss"])

  log_dir = os.path.join(
    'runs', "model_checkpoint")
  tb_logger = TensorboardLogger(log_dir)
  
  checkpoint_handler = ModelCheckpoint(log_dir, 'checkpoint', save_interval=1, n_saved=3)
  trainer.add_event_handler(Events.EPOCH_COMPLETED, checkpoint_handler, {'mymodel': getattr(model, 'module', model)})
  getattr(model, 'module', model).config.to_json_file(os.path.join(log_dir, CONFIG_NAME))
  tokenizer.save_pretrained(log_dir)
  trainer.run(train_loader, max_epochs=10)
  if -1 in [-1, 0] and n_epochs > 0:
    os.rename(os.path.join(log_dir, checkpoint_handler._saved[-1][1]), os.path.join(log_dir, WEIGHTS_NAME))
    tb_logger.close()

In [ ]:
fine_tuning()

/usr/local/lib/python3.7/dist-packages/ignite/handlers/checkpoint.py:971: UserWarning: Argument save_interval is deprecated and should be None. This argument will be removed in 0.5.0.Please, use events filtering instead, e.g. Events.ITERATION_STARTED(every=1000)
  warnings.warn(msg)


[1/5]  20%|##         [00:00<?]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:155: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:1055.)
  exp_avg.mul_(beta1).add_(1.0 - beta1, grad)


63.584983988303065
tensor(4.0713, device='cuda:0')


[1/5]  20%|##         [00:00<?]

...

43.29393097244644
tensor(3.7210, device='cuda:0')


[1/5]  20%|##         [00:00<?]

43.29393097244644
tensor(3.7210, device='cuda:0')


[1/5]  20%|##         [00:00<?]

43.29393097244644
tensor(3.7210, device='cuda:0')


In [ ]:
random.seed(0)
torch.random.manual_seed(0)
torch.cuda.manual_seed(0)

tokenizer_class, model_class = OpenAIGPTTokenizer, OpenAIGPTLMHeadModel
tokenizer = tokenizer_class.from_pretrained("/content/runs/model_checkpoint")
model = model_class.from_pretrained("/content/runs/model_checkpoint")
model.to(device)
add_special_tokens_(model, tokenizer)

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


In [ ]:
personachat_file = cached_path(val_path)
with open(personachat_file, "r", encoding="utf-8") as f:
    dataset = json.loads(f.read())

test_input_text = []
test_labels = []
eval_dict = defaultdict(list)
personalities = []


for key,item in dataset.items():
  if key == 'test':
    test_dict = dataset[key][0]
    for test_key,test_item in test_dict.items():
      if test_key == 'personality':
        personalities.append(test_dict[test_key])
      if test_key == 'utterances':
        for j in range(len(test_dict['utterances'])):
          d = test_dict['utterances'][j]
          for k,i in d.items():
            if k == 'input':
              test_input_text.append(d[k])
            if k == 'label':
              test_labels.append(d[k])
eval_dict['input_text'] = test_input_text
eval_dict['labels'] = test_labels

In [ ]:
print(test_input_text)
print(test_labels)
print(personalities)
print(eval_dict)

['hello , how are you doing tonight ?', 'i am great . i just got back from the club .', 'i would rather eat chocolate cake during this season .', 'i went to club chino . what show are you watching ?', ' do you live in a house or apartment ?']
[' i am well an loving this interaction how are you ?', 'this is my favorite time of the year season wise', 'what club did you go to ? me an timothy watched tv', 'lol oh okay kind of random', 'we watched a show about animals like him']
[['i like getting friends.', ' reptiles make good pets.', 'i love to run around and get out my energy.', 'my family lives on a island.', 'i love watching the leaves change colors.']]
defaultdict(<class 'list'>, {'input_text': ['hello , how are you doing tonight ?', 'i am great . i just got back from the club .', 'i would rather eat chocolate cake during this season .', 'i went to club chino . what show are you watching ?', ' do you live in a house or apartment ?'], 'labels': [' i am well an loving this interaction h

In [ ]:
personality = []
print(personalities)
for i in range(len(personalities[0])):
  personality.append(tokenizer.encode(personalities[0][i]))
print(personality)

[['i like getting friends.', ' reptiles make good pets.', 'i love to run around and get out my energy.', 'my family lives on a island.', 'i love watching the leaves change colors.']]
[[249, 649, 1381, 1662, 239], [39102, 925, 870, 11370, 239], [249, 1119, 485, 1612, 785, 488, 727, 551, 547, 2940, 239], [547, 1463, 2742, 504, 246, 3573, 239], [249, 1119, 1811, 481, 3674, 1796, 5879, 239]]


In [ ]:
no_of_inputs = len(test_input_text)
text = []
output = [] 
B_scores = 0.0
R_scores = 0.0
ROUGE = Rouge()
l1 = []
l2 = []

for i in range(no_of_inputs):
  text.append(tokenizer.encode(test_input_text[i]))
  persona = random.choice(personality)
  with torch.no_grad():
    out_ids = sample_sequence([personality[i]],text, tokenizer, model)
  out_text = tokenizer.decode(out_ids, skip_special_tokens=True)
  text.append(out_ids)
  text = text[-(2+1):]
  print(">>> " +test_input_text[i])
  inp,output = preprocess_text(out_text,test_input_text[i])
  l1.append(inp)
  l2.append(output)
  BLEU_score = sentence_bleu([output],[inp])
  ROUGE_score = ROUGE.get_scores(output,inp)
  print(ROUGE_score)
  B_scores = B_scores + BLEU_score

>>> hello , how are you doing tonight ?
nervous  i love to take chriss car  a  long  x says disc
[{'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.0, 'p': 0.0, 'f': 0.0}}]
>>> i am great . i just got back from the club .
and sir i cant help four youre checking poisons in the hive to protect me i
[{'rouge-1': {'r': 0.2222222222222222, 'p': 0.13333333333333333, 'f': 0.1666666619791668}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.2222222222222222, 'p': 0.13333333333333333, 'f': 0.1666666619791668}}]
>>> i would rather eat chocolate cake during this season .
the second time i came off the that  he let go of a deliberate one for your advanced
[{'rouge-1': {'r': 0.1111111111111111, 'p': 0.058823529411764705, 'f': 0.07692307239644997}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.1111111111111111, 'p': 0.058823529411764705, 'f': 0.07692307239644997}}]
>>> i went to club chino . what show are you watch

In [ ]:
!git clone https://github.com/Tiiiger/bert_score
%cd bert_score
!pip install .

fatal: destination path 'bert_score' already exists and is not an empty directory.
/content/bert_score
Processing /content/bert_score
  Preparing metadata (setup.py) ... done
  Using cached transformers-4.19.1-py3-none-any.whl (4.2 MB)
  Using cached tokenizers-0.12.1-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (6.6 MB)
  Created wheel for bert-score: filename=bert_score-0.3.11-py3-none-any.whl size=60607 sha256=e544b9d3201ca24042742576c0c04c5b66586e28820d4c05dc7bc8494a4b7fb3
  Stored in directory: /tmp/pip-ephem-wheel-cache-db0ffry8/wheels/a7/12/f8/8da4f6ac6ff005e9a0202402458a4b37dc94d267cd0624459d
Successfully built bert-score
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.5.2
    Uninstalling tokenizers-0.5.2:
      Successfully uninstalled tokenizers-0.5.2
  Attempting uninstall: transformers
    Found existing installation: transformers 2.5.1
    Uninstalling transformers-2.5.1:
      Successfully uninstalled transformers-2.5.1
  Att

In [ ]:
BLEU_score = sentence_bleu(l2,l1)

In [ ]:
print(l2)
print(l1)

['nervous  i love to take chriss car  a  long  x says disc', 'and sir i cant help four youre checking poisons in the hive to protect me i', 'the second time i came off the that  he let go of a deliberate one for your advanced', 'with a dancing images  while parker grew younger and he had a few birthdays he broke', 'arm sun coming later adventures continued  his eyes fondly watching one strike and averting one']
['hello  how are you doing tonight', 'i am great  i just got back from the club', 'i would rather eat chocolate cake during this season', 'i went to club chino  what show are you watching', 'do you live in a house or apartment']


In [ ]:
scorer = score.BleurtScorer()
scores = scorer.score(references=l2, candidates=l1)
print(scores)

INFO:tensorflow:No checkpoint specified, defaulting to BLEURT-tiny.
INFO:tensorflow:Reading checkpoint /usr/local/lib/python3.7/dist-packages/bleurt/test_checkpoint.
INFO:tensorflow:Config file found, reading.
INFO:tensorflow:Will load checkpoint dbleurt_tiny
INFO:tensorflow:Loads full paths and checks that files exists.
INFO:tensorflow:... name:dbleurt_tiny
INFO:tensorflow:... vocab_file:vocab.txt
INFO:tensorflow:... bert_config_file:bert_config.json
INFO:tensorflow:... do_lower_case:True
INFO:tensorflow:... max_seq_length:512
INFO:tensorflow:Creating BLEURT scorer.
INFO:tensorflow:Creating WordPiece tokenizer.
INFO:tensorflow:WordPiece tokenizer instantiated.
INFO:tensorflow:Creating Eager Mode predictor.
INFO:tensorflow:Loading model.
INFO:tensorflow:BLEURT initialized.
[-1.4691402912139893, -1.2327680587768555, -1.4253696203231812, -1.214280366897583, -1.4594740867614746]


# References:

https://medium.com/huggingface/how-to-build-a-state-of-the-art-conversational-ai-with-transfer-learning-2d818ac26313

https://huggingface.co/docs/transformers/index

https://github.com/huggingface/transfer-learning-conv-ai

https://pytorch.org/ignite/engine.html